In [1]:
import numpy as np
import uproot, json

# from util.matrix import loadMatrices


In [2]:
def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result

In [3]:
def getRot( apparent, actual):
    """
    computes rotation from A to B when rotated through origin.
    shift A and B before, if rotation did not already occur through origin!

    see https://math.stackexchange.com/a/476311
    or https://en.wikipedia.org/wiki/Cross_product
    and https://en.wikipedia.org/wiki/Rotation_matrix#Conversion_from_rotation_matrix_and_to_axis%E2%80%93angle

    This function works on 3D points only, do not give homogeneous coordinates to this!
    """
    # error handling
    if np.linalg.norm(apparent) == 0 or np.linalg.norm(actual) == 0:
        self.logger.log("\nERROR. can't create rotation with null vector!\n")
        return

    # assert shapes
    assert apparent.shape == actual.shape

    # normalize vectors
    apparent = apparent / np.linalg.norm(apparent)
    actual = actual / np.linalg.norm(actual)

    # calc rot angle by dot product
    cosine = np.dot(apparent, actual)  # cosine

    # make 2D vectors so that transposing works
    cVector = apparent[np.newaxis].T
    dVector = actual[np.newaxis].T

    # compute skew symmetric cross product matrix
    crossMatrix = (dVector @ cVector.T) - (cVector @ dVector.T)

    # compute rotation matrix
    R = (
        np.identity(3)
        + crossMatrix
        + np.dot(crossMatrix, crossMatrix) * (1 / (1 + cosine))
    )

    return R

def rotationMatrixToEulerAngles(R):

    assert R.shape == (4, 4) or R.shape == (3, 3)

    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])


In [4]:
def quantileCut(xyzArray, cut=4):

    if cut == 0:
        return xyzArray

    # calculate cut length
    cut = int(len(xyzArray) * (cut / 100))

    # calculate center of mass (where most points are)
    # don't use average, some values are far too large, median is a better estimation
    comMed = np.median(xyzArray[:, 3:6], axis=0)

    # now, sort by distance and cut largest
    # first, calculate distace of each point to center of mass
    distVec = xyzArray[:, 3:6] - comMed

    # calculate length of these distance vectors
    distVecNorm = np.linalg.norm(distVec, axis=1)

    # sort the entire array by this length
    xyzArray = xyzArray[distVecNorm.argsort()]

    # cut the largest values
    resxyzArrayxyzArray = xyzArray[:-cut]

    return resxyzArrayxyzArray


def getIPfromRootFiles(filename, maxNoOfFiles=0):
    # fileTree = uproot.open(filename)['pndsim']

    # make empty 2D (n times 4) result array for each individual IP position (that's per file)
    IPs = np.empty((0, 4))

    runIndex = 0
    for array in uproot.iterate(
        filename,
        [
            "LMDTrackQ.fTrkRecStatus",
            "LMDTrackQ.fXrec",
            "LMDTrackQ.fYrec",
            "LMDTrackQ.fZrec",
        ],
        library="np",
        allow_missing=True,
    ):

        recStat = np.concatenate(array["LMDTrackQ.fTrkRecStatus"]).ravel()
        recX = np.concatenate(array["LMDTrackQ.fXrec"]).ravel()
        recY = np.concatenate(array["LMDTrackQ.fYrec"]).ravel()
        recZ = np.concatenate(array["LMDTrackQ.fZrec"]).ravel()

        # apply mask for correctly reconstructed track and tracks within 5cm
        # that means reconstructed IP must be within 5cm of 0 in all directions
        thresh = 5
        mask = (
            (recStat == 0)
            & (np.abs(recX) < thresh)
            & (np.abs(recY) < thresh)
            & (np.abs(recZ) < thresh)
        )

        recXmask = recX[mask]
        recYmask = recY[mask]
        recZmask = recZ[mask]

        # don't worry, this is done by reference, nothing is copied here
        outarray = np.array([recXmask, recYmask, recZmask]).T

        outarray = quantileCut(outarray, 4)

        foundIP = np.average(outarray, axis=0)
        resultIPhomogeneous = np.ones(4)
        resultIPhomogeneous[:3] = foundIP
        # print(f"loaded {len(outarray)} tracks")
        # print(f"found ip: {resultIPhomogeneous}")
        IPs = np.vstack((IPs, resultIPhomogeneous))
        runIndex += 1
        if runIndex == maxNoOfFiles:
            break

    print(f"read {runIndex} file(s)")
    return np.average(IPs, axis=0)


In [5]:
np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

idealDetectorMatrixPath = "../../input/detectorMatricesIdeal.json"
idealDetectorMatrices = loadMatrices(idealDetectorMatrixPath)

# ipActual = np.array([-0.023392, 0.00718, 0.004023, 1.0])

# rootFileWildcard = "comparisonData/box100/rawData/Lumi_TrksQA_*.root:pndsim"
path = "/mnt/himsterData/roklasen/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/geo_misalignmentmisMat-box/100000/1-100_uncut/no_alignment_correction/"
rootFileWildcard = "Lumi_TrksQA_*.root:pndsim"
IPfromLMD = getIPfromRootFiles(path+rootFileWildcard,2)
print(f'found this ip: {IPfromLMD}')
ipStefanFit = [-0.12169, -0.107023, 0, 1]
ipApparent = IPfromLMD
print(f'comparison, IP from gauss fit: {ipStefanFit}')


# we want the rotation of the lumi box, so we have to change the basis
matPndtoLmd = idealDetectorMatrices["/cave_1/lmd_root_0"]
zero = [0, 0, 0, 1]

# perform passive transformation of these points to the system
# of the LMD, so that the rotation occurs arund it's origin
zeroAt = (np.linalg.inv(matPndtoLmd) @ zero)[:3]
ipApparentLMD = (np.linalg.inv(matPndtoLmd) @ ipApparent)[:3]

print(f"zero at: {zeroAt}\nipApparent: {ipApparentLMD}")

#! order is: IP_from_LMD, IP_actual (i.e. from PANDA)
Ra = getRot(ipApparentLMD, zeroAt)

# homogenize the matrix again
R1 = np.identity(4)
R1[:3, :3] = Ra

# homogenize the matrix again
R1 = np.identity(4)
R1[:3, :3] = Ra

# after that, add the remaining translation (direct shift towards IP), not implemented yet
resultJson = {"/cave_1/lmd_root_0": R1}


FileNotFoundError: file not found

    '/mnt/himsterData/roklasen/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/geo_misalignmentmisMat-box/100000/1-100_uncut/no_alignment_correction/Lumi_TrksQA_*.root:pndsim'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in str/

In [ ]:
mMis = loadMatrices("../../output/misMat-box.json")[
    "/cave_1/lmd_root_0"
]

print("----------------")
print("Found Euler Angles:")
print(rotationMatrixToEulerAngles(R1)*1e3)
print("----------------")
print("Actual Euler Angles:")
print(rotationMatrixToEulerAngles(mMis)*1e3)


: 

: 